In [1]:
import sys
# import pytest

import sys
sys.path.append("../")

from mpa_sim.services.tools import Calculator

In [2]:
PARAMETERS = {
    'iterations': 20,
    'seed_value1': 1,
    'seed_value2': 2,
    'indoor_users_percentage': 50,
    'los_breakpoint_m': 500,
    'tx_macro_baseline_height': 30,
    'tx_macro_power': 40,
    'tx_macro_gain': 16,
    'tx_macro_losses': 1,
    'tx_micro_baseline_height': 10,
    'tx_micro_power': 24,
    'tx_micro_gain': 5,
    'tx_micro_losses': 1,
    'rx_gain': 4,
    'rx_losses': 4,
    'rx_misc_losses': 4,
    'rx_height': 1.5,
    'building_height': 5,
    'street_width': 20,
    'above_roof': 0,
    'network_load': 50,
    'percentile': 50,
    'sectorization': 3,
    'mnos': 2,
    'asset_lifetime': 10,
    'discount_rate': 3.5,
    'opex_percentage_of_capex': 10,
    'signaling_overhead': 0.18, # overhead for control channels
    'modulation_compression': True,
    'ru_du_ratio': 3,
    'du_cuup_ratio': 1,
    'cucp_cuup_ratio': 1,
    'compression_ratio': {
        'QPSK': 0.06,
        '16QAM': 0.12,
        '64QAM': 0.18,
        '256QAM': 0.25,
    },
    'Numerology': 0, #µ(0)=15kHz, µ(1)=30kHz, µ(2)=60kHz
    'Bandwidth': 20, #MHz (BW(j))
    'Modulation': 256, #QAM
    # 'Number_of_cells': 4,
    'Number_of_carriers': 1,
    'Number_of_UEs': 64, # per cell
    'DL': 1544, # Mbps per cell
    'UL': 824, # Mbps per cell
    'number_of_aggregated_component_carriers': 1, # (J) max = 16
    'DL_MIMO_Layers': 2, # (v(j)) max = 8 
    'UL_MIMO_Layers': 1, # (v(j)) max = 4
    'MU_MIMO':  1, # Number of Beam with MU-MIMO Users; 
    'Number_of_logical_antenna_ports': 8, # 1/2/4/8/16. Used for fronthaul throughput only, should be higher than MIMO Layers
    'Rmax': 0.92578125, # value depends on the type of coding from 3GPP 38.212 and 3GPP 38.214 (For LDPC code maximum number is 948/1024) 
    'spectral_efficiency': 5.1152, #3GPP TS 38.214 version 15.3.0 Release 15, Table 5.1.3.1-2 (64QAM)
    'Scaling_factor' : 1, # (f(j))
    'DL_UP_ratio': 0.7, #10/14, # based on slot configuration 6:4:4 (6 dl, 4 guard, 4 ul) ref: https://jira.cec.lab.emc.com/browse/MP-4406
    'IQ_mantissa_bitwidth': 8, # per I or per Q
    'IQ_exp_bitwidth': 4,
}

In [3]:
COSTS = {
    #all costs in $USD
    'cpu': [
        {
            'model': '6148',
            'price': 2988.39,
            'cores': 20,
            'power': 150,
        },
        {
            'model': '6230',
            'price': 1582.72,
            'cores': 26,
            'power': 150,
        },
    ],
    'switch': [
        {
            'model': '10GB Fiber',
            'price': 2000,
            'ports': 8,
            'power': 150,
            'speed': 10, #Gbps
        },
        {
            'model': '25GB Fiber',
            'price': 8000,
            'ports': 8,
            'power': 350,
            'speed': 25, #Gbps
        },
    ],
    'server': [
        {
            'model': 'XR11',
            'price': 900,
            'size': '1U',
            'power': 150,
            'cpu_max': 2,
            'pcie_max': 4,
        },
        {
            'model': 'R750',
            'price': 1700,
            'size': '2U',
            'power': 200,
            'cpu_max': 2,
            'pcie_max': 4,
        },
    ],
    'lease_line_installation': [
        {
            'model': '1Gbps',
            'price': 1000,
            'speed': 1, #Gbps
        },
        {
            'model': '10Gbps',
            'price': 2000,
            'speed': 10, #Gbps
        },        
        {
            'model': '100Gbps',
            'price': 10000,
            'speed': 100, #Gbps
        },
    ],
    'lease_line_installation_per_meter': [
        {
            'model': '1Gbps',
            'price': 0.1,
            'speed': 1, #Gbps
        },
        {
            'model': '10Gbps',
            'price': 0.2,
            'speed': 10, #Gbps
        },        
        {
            'model': '100Gbps',
            'price': 0.3,
            'speed': 100, #Gbps
        },
    ],
    'lease_line_rental': [
        {
            'model': '1Gbps',
            'price': 500,
            'speed': 1, #Gbps
        },
        {
            'model': '10Gbps',
            'price': 5000,
            'speed': 10, #Gbps
        },        
        {
            'model': '100Gbps',
            'price': 15000,
            'speed': 100, #Gbps
        },
    ],
    'sfp': [
        {
            'model': 'QSPF28',
            'price': 599, #usd
            'distance': 10000, #meters
            'speed': 100, #Gbps
        },
        {
            'model': 'SPF28',
            'price': 420, #usd
            'distance': 30000, #meters
            'speed': 25, #Gbps
        },
        {
            'model': 'Dell SPF',
            'price': 115, #usd
            'distance': 300, #meters
            'speed': 10, #Gbps
        },
    ],
    'single_sector_antenna': [
        {
            'price': 1500
        },
    ],
    'ru': [
        {
            'price': 4000
        },
    ],
    'du': [
        {
            'price': 10000
        },
    ],
    'cu': [
        {
            'price': 15000
        },
    ],
    'tower': [
        {
            'price': 10000
        },
    ],
    'transportation': [
        {
            'price': 10000
        },
    ],
    'installation': [
        {
            'price': 5000
        },
    ],
    'site_rental': [
        {
            'price': 8000
        },
    ],
    'power_generator_battery_system': [
        {
            'price': 5000
        },
    ],
    'smo': [
        {
            'price': 2000
        },
    ],    
    'l1_controller': [
        {
            'model': 'marvell',
            'price': 900,
            'ports': 4,
            'speed': 6, # Gbps
        },
    ],

}



In [4]:
cal = Calculator(simulation_parameters = PARAMETERS)

### Maximum Cell Throughput

In [5]:
mode = 'FDD'

In [6]:
tput = cal.max_cell_throughput(mode = mode)
tput

{'DL': 0.21630041999999994, 'UL': 0.11832461999999996}

In [7]:
total = tput['DL'] + tput['UL']
total

0.33462503999999993

### ECPRI Throughput

In [8]:
cal.ecpri_throughput() #Gbps

3.3621503999999995

### DU Traffic

In [9]:
cal.du_nb_traffic(mode=mode)

0.6489012599999998

### CU-UP Traffic

In [10]:
cal.cuup_nb_traffic(mode=mode)

0.5320990331999998

### CU-CP Traffic

In [11]:
cal.cucp_nb_traffic(mode=mode)

0.11680222679999996

In [12]:
cal.core_traffic(mode=mode)

0.5320990331999998

In [13]:
cal.smo_traffic(mode=mode)

0.11680222679999996

## Required Server Resources

In [14]:
from mpa_sim.services.hw_estimator import CPU_forecasting_model
cpu_fm = CPU_forecasting_model(simulation_parameters = PARAMETERS)

In [15]:
PARAMETERS["DL"] = tput['DL']
PARAMETERS["UL"] = tput['UL']

In [16]:
model_CU, model_DU, scaler = cpu_fm.loadModels()
res = cpu_fm.estimateCPU_Utilization_CU_DU(model_CU, model_DU, scaler)
res

c:\Python39\lib\site-packages\sklearn\base.py:329: UserWarning: Trying to unpickle estimator LinearRegression from version 0.24.2 when using version 1.0.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  warnings.warn(
c:\Python39\lib\site-packages\sklearn\base.py:329: UserWarning: Trying to unpickle estimator MinMaxScaler from version 0.24.2 when using version 1.0.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  warnings.warn(
c:\Python39\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but MinMaxScaler was fitted without feature names
  warnings.warn(
c:\Python39\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but MinMa

{'DU_utils': 125.92865995564433,
 'CUCP_utils': 194.7727762431615,
 'CUUP_utils': 194.81405784642513}

In [17]:
import pickle as pl

In [18]:
fi = open("data_res.plk", 'wb')
pl.dump(res, fi)
fi.close()